In [1]:
import pandas as pd
import random
import matplotlib.pyplot as plt

from scipy.spatial.distance import cdist

from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

import numpy as np
from deap import base
from deap import tools
from deap import creator
from deap.algorithms import eaSimple

In [2]:
features = ['Soil_Quality', 'Seed_Variety', 'Fertilizer_Amount_kg_per_hectare', 'Sunny_Days', 'Rainfall_mm', 'Irrigation_Schedule']
target = ['Yield_kg_per_hectare']

df_teste = pd.read_csv("agricultural_yield_test.csv")
df_treino = pd.read_csv("agricultural_yield_train.csv")

df_teste = df_teste.reindex(features + target, axis=1) 
df_teste = df_teste.dropna()

df_treino = df_treino.reindex(features + target, axis=1)
df_treino = df_treino.dropna()

In [3]:
X_treino_floresta = df_treino.reindex(features, axis=1).values
y_treino_floresta = df_treino.reindex(target, axis=1).values.ravel()

X_teste_floresta = df_teste.reindex(features, axis=1).values
y_teste_floresta = df_teste.reindex(target, axis=1).values.ravel()

In [4]:
X_treino_floresta

array([[ 96.4156574 ,   1.        , 147.85304019,  94.59392594,
        444.26756885,   3.        ],
       [ 92.35262586,   0.        , 281.56539608,  90.5046436 ,
        517.58549117,   7.        ],
       [ 63.71478519,   1.        , 137.86493988,  97.32934017,
        420.3109448 ,   8.        ],
       ...,
       [ 67.47848719,   1.        , 120.01712248, 102.30862733,
        514.3727475 ,   8.        ],
       [ 85.17689046,   1.        , 247.84738886, 114.95663392,
        695.03580096,   6.        ],
       [ 87.8382803 ,   0.        , 244.21653603,  98.31373807,
        590.79491539,   6.        ]])

In [5]:
y_treino_floresta

array([683.75911882, 678.71486056, 934.69197491, ..., 932.82956077,
       889.7384376 , 554.69838897])

In [6]:
X_teste_floresta

array([[ 93.30472076,   0.        , 132.52221761,  96.67092181,
        602.38623718,   3.        ],
       [ 83.67465324,   1.        ,  57.28399706,  99.00755589,
        466.51825061,   8.        ],
       [ 65.96303292,   1.        , 227.89547903, 104.8442723 ,
        510.32049549,   4.        ],
       ...,
       [ 94.14552167,   1.        , 196.47370358, 110.00347356,
        458.24140783,   3.        ],
       [ 61.45212028,   1.        ,  80.61835714,  97.81551671,
        778.6525395 ,   4.        ],
       [ 55.19590628,   0.        , 122.47323884,  90.31172043,
        582.99900253,   4.        ]])

In [7]:
y_teste_floresta

array([278.98656345, 836.43484043, 785.88178724, ..., 776.01363423,
       502.31948382, 257.03254443])

In [8]:
qualidade_solo = list(df_teste['Soil_Quality'])
variedade_semente = list(df_teste['Seed_Variety'])
fertilizante = list(df_teste['Fertilizer_Amount_kg_per_hectare'])
dias_de_sol = list(df_teste['Sunny_Days'])
chuva = list(df_teste['Rainfall_mm'])
irrigação = list(df_teste['Irrigation_Schedule'])

rendimento = list(df_teste['Yield_kg_per_hectare'])

In [9]:
lista = [qualidade_solo, variedade_semente, fertilizante, dias_de_sol, chuva, irrigação]

min_max = []

for i in lista:
    min_max.append(min(i))
    min_max.append(max(i))

print("Tamanho lista:", len(min_max)) # Deve ser 6 x 2
print(min_max)

Tamanho lista: 12
[50.00362248192461, 99.99889758903616, 0, 1, 50.06265491148749, 299.9920540908917, 66.72446890908668, 138.52020171941672, 102.0075178449252, 876.694216751489, 0, 15]


### Floresta Aleatória

In [10]:
SEMENTE_ALEATORIA = 61455

modelo_fa = RandomForestRegressor(random_state=SEMENTE_ALEATORIA)

In [11]:
modelo_fa.fit(X_treino_floresta, y_treino_floresta)

y_verdadeiro = y_teste_floresta
y_previsao = modelo_fa.predict(X_teste_floresta)
    
RMSE = mean_squared_error(y_verdadeiro, y_previsao, squared=False)

print(f"O RMSE do modelo árvore de decisão foi de {RMSE} unidades de y.")

O RMSE do modelo árvore de decisão foi de 54.89847461072623 unidades de y.


Testando o modelo para o maior rendimento do data frame de teste

In [12]:
num = max(rendimento)

indice = rendimento.index(num)

linha = indice
individuo = df_treino.iloc[linha,:-1].values

print(individuo)
print(rendimento[indice])

[ 51.99126884   0.          66.6478779  100.72440418 454.76365482
   4.        ]
1406.1107054578508


In [30]:
print(individuo)
y_previsto = modelo_fa.predict([individuo])

print(y_previsto)

[ 51.99126884   0.          66.6478779  100.72440418 454.76365482
   4.        ]
[384.92875313]


### Usando módulo DEAP para algoritmo genético

In [14]:
TAMANHO_POPULACAO = 100
NUM_GERACOES = 150
CHANCE_DE_CRUZAMENTO = 0.5
CHANCE_DE_MUTACAO = 0.5
TAMANHO_TORNEIO = 5
TAMANHO_HALL_DA_FAMA = 1

In [15]:
toolbox = base.Toolbox()

In [16]:
i = 0
j = 0

for _ in range(6):
    print(features[i], '-> Min =', min_max[j], '/ Max =', min_max[j+1])
    print()
    i += 1
    j += 2

Soil_Quality -> Min = 50.00362248192461 / Max = 99.99889758903616

Seed_Variety -> Min = 0 / Max = 1

Fertilizer_Amount_kg_per_hectare -> Min = 50.06265491148749 / Max = 299.9920540908917

Sunny_Days -> Min = 66.72446890908668 / Max = 138.52020171941672

Rainfall_mm -> Min = 102.0075178449252 / Max = 876.694216751489

Irrigation_Schedule -> Min = 0 / Max = 15



In [17]:
print(min_max)

[50.00362248192461, 99.99889758903616, 0, 1, 50.06265491148749, 299.9920540908917, 66.72446890908668, 138.52020171941672, 102.0075178449252, 876.694216751489, 0, 15]


In [18]:
def gene_soil_quality():
    return random.uniform(min_max[0], min_max[1])

def gene_seed_variety():
    return random.choice([min_max[2], min_max[3]])

def gene_fertilizer_amount():
    return random.uniform(min_max[4], min_max[5])

def gene_sunny_days():
    return random.uniform(min_max[6], min_max[7])

def gene_rainfall():
    return random.uniform(min_max[8], min_max[9])

def gene_irrigation_schedule():
    return random.randint(int(min_max[10]), int(min_max[11]))

Registrando os genes no toolbox

In [19]:
toolbox.register("gene_soil_quality", gene_soil_quality)
toolbox.register("gene_seed_variety", gene_seed_variety)
toolbox.register("gene_fertilizer_amount", gene_fertilizer_amount)
toolbox.register("gene_sunny_days", gene_sunny_days)
toolbox.register("gene_rainfall", gene_rainfall)
toolbox.register("gene_irrigation_schedule", gene_irrigation_schedule)

In [20]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individuo", list, fitness=creator.FitnessMax)

In [21]:
def cria_individuo():
    return creator.Individuo([
        toolbox.gene_soil_quality(),
        toolbox.gene_seed_variety(),
        toolbox.gene_fertilizer_amount(),
        toolbox.gene_sunny_days(),
        toolbox.gene_rainfall(),
        toolbox.gene_irrigation_schedule()
    ])

In [22]:
individuo_teste = cria_individuo()

print(individuo_teste)

[73.43763415836912, 1, 277.2308958233448, 123.40257405700751, 541.1484509237449, 8]


Criando e testando a função objetivo

In [23]:
def mutacao_personalizada(individuo, chance_de_mutacao, min_max_list):
    
    min_max = min_max_list
        
    for gene in range(len(individuo)):
        if random.random() < chance_de_mutacao:
            if gene == 0:
                individuo[gene] = random.uniform(min_max[0], min_max[1])
            elif gene == 1:
                individuo[gene] = 0 if individuo[gene] == 1 else 1
            elif gene == 2:
                individuo[gene] = random.uniform(min_max[4], min_max[5])
            elif gene == 3:
                individuo[gene] = random.uniform(min_max[6], min_max[7])
            elif gene == 4:
                individuo[gene] = random.uniform(min_max[8], min_max[9])
            elif gene == 5:
                individuo[gene] = random.randint(int(min_max[10]), int(min_max[11]))
        
    return (individuo, )

In [31]:
def funcao_objetivo_floresta(individuo):
    
    # Realizar a predição
    individuo = np.array(individuo).reshape(1, -1)  # Adicionar dimensão de batch
    y_previsao = modelo_fa.predict(individuo)
    
    # Calcular a distância de Manhattan entre y_pred e todos os valores de y_true 
    y_prev_val = y_previsao[0]
    y_true_vals = y_teste_floresta.reshape(-1, 1)
    distances = cdist([[y_prev_val]], y_true_vals, metric='cityblock')
    
    # Verificar a menor distância
    min_distance = distances.min()
    if min_distance > 500:
        y_prev_val = 0
    
    return (y_prev_val, )

In [32]:
toolbox.register("evaluate", funcao_objetivo_floresta)

In [33]:
toolbox.register("cria_individuo", cria_individuo)
toolbox.register("populacao", tools.initRepeat, list, toolbox.cria_individuo, TAMANHO_POPULACAO)
toolbox.register("select", tools.selTournament, tournsize=TAMANHO_TORNEIO)
toolbox.register("mate", tools.cxUniform, indpb=0.5)

toolbox.register(
    "mutate",
    mutacao_personalizada,
    chance_de_mutacao = CHANCE_DE_MUTACAO,
    min_max_list = min_max
)

hall_da_fama_FA = tools.HallOfFame(TAMANHO_HALL_DA_FAMA)

estatisticas_FA = tools.Statistics(lambda ind: ind.fitness.values)
estatisticas_FA.register("média", np.mean)
estatisticas_FA.register("desv. padrão", np.std)
estatisticas_FA.register("min", np.min)
estatisticas_FA.register("max", np.max)

populacao_inicial_FA = toolbox.populacao()

In [34]:
populacao_final_FA, log_FA = eaSimple(
    populacao_inicial_FA,
    toolbox,
    cxpb=CHANCE_DE_CRUZAMENTO,
    mutpb=CHANCE_DE_MUTACAO,
    ngen=NUM_GERACOES,
    stats=estatisticas_FA,
    halloffame=hall_da_fama_FA,
    verbose=True,
)

gen	nevals	média  	desv. padrão	min    	max    
0  	100   	775.671	296.134     	124.631	1323.99
1  	69    	1008.8 	253.989     	167.815	1323.99
2  	76    	1048.31	266.792     	206.534	1328.42
3  	71    	1129.21	254.694     	153.394	1329.38
4  	70    	1139.75	224.77      	487.712	1329.38
5  	68    	1122.93	266.779     	359.194	1329.38
6  	73    	1176.11	227.775     	384.037	1333.1 
7  	81    	1141.6 	254.334     	233.973	1336.61
8  	72    	1131.31	280.916     	284.761	1338.82
9  	78    	1138.53	249.639     	321.815	1341.36
10 	86    	1130.35	265.031     	454.653	1347.41
11 	72    	1174.81	273.929     	211.398	1347.41
12 	80    	1140.21	293.655     	309.082	1348.36
13 	67    	1154.89	250.182     	362.028	1348.36
14 	81    	1188.51	232.891     	349.219	1348.36
15 	71    	1179.61	229.309     	328.217	1348.36
16 	84    	1140.57	244.738     	413.457	1348.36
17 	76    	1184.14	241.234     	383.856	1348.36
18 	61    	1231.46	196.691     	495.727	1348.36
19 	70    	1176.85	234.13      	485.785	

In [35]:
print(hall_da_fama_FA.items)

[[69.96944903888709, 1, 298.0520810373879, 115.79260030219567, 164.80241599918014, 13]]


In [36]:
hall_da_fama_FA = np.array(hall_da_fama_FA).reshape(1, -1)
print(hall_da_fama_FA)
y_previsao = modelo_fa.predict(hall_da_fama_FA)
    
print(f"A previsão máxima de rendimento encontrada foi: {y_previsao[0]}")

[[ 69.96944904   1.         298.05208104 115.7926003  164.802416
   13.        ]]
A previsão máxima de rendimento encontrada foi: 1351.8934062229985
